In [3]:
# "interesting" parameter IDs identified by Pavan and Meghan
targets = [ "t73", "t58", "t121", "t122", "t116", "t118", "t115",
 "t83", "t82", "t74", "t119", "t72", "t71", "t120", "t166", "t132",
 "t133", "t130", "t143", "t59", "t60", "t62", "t61", "t127", "t157",
 "t167", "t142",
]
len(targets)

27

In [4]:
from openff.toolkit import ForceField

sage20 = ForceField("openff-2.0.0.offxml")
sage21 = ForceField("openff-2.1.0.offxml")

pavan and meghan identified these interesting parameters 
based on Sage 2.0.0, so my first step was porting their
force field changes from 2.0 to 2.1. I did this automatically
just by looking at parameter ID sets between pavan and 2.0 and
pavan and 2.1, but I think this was too simplistic.

It's not enough to compare IDs and hope they are the same. It's not
even really enough to compare SMIRKS patterns as I tried in my first
attempt. We need to make sure things like the phase and periodicity
are also the same, but changes to the parameter values (k) have to
be allowed.

In [5]:
tors20 = sage20.get_parameter_handler("ProperTorsions")
tors21 = sage21.get_parameter_handler("ProperTorsions")

In [10]:
int_tors20 = {t.id: t for t in tors20 if t.id in targets}
int_tors21 = {t.id: t for t in tors21 if t.id in targets}

In [7]:
len(int_tors20)

27

In [8]:
len(int_tors21)

27

Despite what I wrote above, an initial search for different smirks
does make sense.

In [13]:
for id, param in int_tors20.items():
    smirks20 = param.smirks
    smirks21 = int_tors21[id].smirks
    if smirks20 != smirks21:
        print(id)
        print(smirks20)
        print(smirks21)

t130
[*:1]-[#7X4,#7X3:2]-[#7X4,#7X3:3]-[*:4]
[*:1]~[#7X4,#7X3:2]-[#7X4,#7X3:3]~[*:4]


now let's see if stripping out the k parameters is enough to get equality

In [23]:
int_tors20["t73"].k = []
int_tors21["t73"].k = []
print(int_tors20["t73"] == int_tors21["t73"])
int_tors20["t73"], int_tors21["t73"]

False


(<ProperTorsionType with smirks: [*:1]~[#7X3,#7X2-1:2]-[#6X3:3]~[*:4]  periodicity1: 2  phase1: 180.0 degree  id: t73  idivf1: 1.0  >,
 <ProperTorsionType with smirks: [*:1]~[#7X3,#7X2-1:2]-[#6X3:3]~[*:4]  periodicity1: 2  phase1: 180.0 degree  id: t73  idivf1: 1.0  >)

not sure why this doesn't work, probably something that isn't printed...
just manually check the fields that we care about: id, smirks, periodicity, phase, and idivf

In [27]:
def eq(p1, p2):
    return p1.id == p2.id and p1.smirks == p2.smirks and p1.periodicity == p2.periodicity and p1.phase == p2.phase and p1.idivf == p2.idivf

In [28]:
eq(int_tors20["t73"], int_tors21["t73"])

True

In [30]:
for id, param in int_tors20.items():
    smirks20 = param
    smirks21 = int_tors21[id]
    if not eq(smirks20, smirks21):
        print(id)

t130
t143
t157


now we have some IDs to look more closely at

In [32]:
def diff(p1, p2):
    for attr in ["id", "smirks", "periodicity", "phase", "idivf"]:
        if getattr(p1, attr) != getattr(p2, attr):
            return attr
    return None

In [33]:
diff(int_tors20["t130"], int_tors21["t130"])

'smirks'

In [34]:
def qdiff(pid): return diff(int_tors20[pid], int_tors21[pid])

In [35]:
qdiff("t130")

'smirks'

In [36]:
qdiff("t143")

'periodicity'

In [37]:
qdiff("t157")

'periodicity'

In [39]:
int_tors20["t143"]

<ProperTorsionType with smirks: [*:1]~[#16X4,#16X3+0:2]-[#7:3]~[*:4]  periodicity1: 1  phase1: 0.0 degree  id: t143  k1: 2.229703816414 kilocalorie / mole  idivf1: 1.0  >

In [40]:
int_tors21["t143"]

<ProperTorsionType with smirks: [*:1]~[#16X4,#16X3+0:2]-[#7:3]~[*:4]  periodicity1: 1  periodicity2: 2  periodicity3: 3  phase1: 180.0 degree  phase2: 0.0 degree  phase3: 0.0 degree  id: t143  k1: -1.6882659825 kilocalorie / mole  k2: 0.3191499888753 kilocalorie / mole  k3: 0.2193673170111 kilocalorie / mole  idivf1: 1.0  idivf2: 1.0  idivf3: 1.0  >

# Conclusion
These changes are all fairly simple and described in the Sage 2.1.0 readme. t130 had a small smirks change
that doesn't affect the multiplicity stuff, and t143 and t157 had additional periodicities added. In every case
it should be safe to take the Sage 2.1.0 parameter as the base value and start splitting from there

# Porting changes

With that out of the way, let's see which changes Pavan and Meghan already included in their force field.

In [42]:
pavan = ForceField("01_generate-forcefield/force-field.offxml", allow_cosmetic_attributes=True)

In [59]:
torspv = {t.id for t in pavan.get_parameter_handler("ProperTorsions")}
tors20 = {t.id for t in sage20.get_parameter_handler("ProperTorsions")}
torspv - tors20

{'t116a',
 't116b',
 't116c',
 't118a',
 't121a',
 't122a',
 't122b',
 't122c',
 't122d',
 't122e',
 't122f',
 't130a',
 't130b',
 't130c',
 't130d',
 't132a',
 't132b',
 't132c',
 't132d',
 't133a',
 't142a',
 't142b',
 't142c',
 't142d',
 't142e',
 't142f',
 't143a',
 't143b',
 't143c',
 't143d',
 't143e',
 't143f',
 't157a',
 't73a',
 't74a',
 't82a',
 't83a'}

In [54]:
tors20 - torspv

{'t122', 't130', 't132', 't142', 't143'}

I was actually expecting this set to be empty. Apparently in some cases they deleted the parent parameter, while in other cases 
(most cases) they preserved the parent and added child parameters

What I want to do here is go through each of the interesting parameters and split them by hand. Unfortunately, looking
at the XML files directly is very noisy, so I will probably work on some shell scripts to pull out the interesting parts
of the lines.

In [58]:
torspv = {t.id: t.smirks for t in pavan.get_parameter_handler("ProperTorsions")}
torspv["t58"]

KeyError: 't58a'